# 4. 깊은 순환 신경망(Deep Recurrent Neural Network)

앞서 RNN도 다수의 은닉층을 가질 수 있다고 언급한 바 있습니다. 위의 그림은 순환 신경망에서 은닉층이 1개 더 추가되어 은닉층이 2개인 깊은(deep) 순환 신경망의 모습을 보여줍니다. 위의 코드에서 첫번째 은닉층은 다음 은닉층에 모든 시점에 대해서 은닉 상태 값을 다음 은닉층으로 보내주고 있습니다.

깊은 순환 신경망을 파이토치로 구현할 때는 nn.RNN()의 인자인 num_layers에 값을 전달하여 층을 쌓습니다. 층이 2개인 깊은 순환 신경망의 경우, 앞서 실습했던 임의의 입력에 대해서 출력이 어떻게 달라지는지 확인해봅시다.

In [7]:
import torch
import torch.nn as nn


input_size = 5 # 입력의 크기
hidden_size = 8 # 은닉 상태의 크기

# (batch_size, time_steps, input_size)
inputs = torch.Tensor(1, 10, 5)
cell = nn.RNN(input_size = 5, hidden_size = 8, num_layers = 2, batch_first=True)
outputs, _status = cell(inputs)
print(outputs.shape) # 모든 time-step의 hidden_state
print(outputs)

torch.Size([1, 10, 8])
tensor([[[ 0.2691,  0.4047,  0.7810,  0.1488, -0.3273,  0.1126, -0.6672,
          -0.5955],
         [-0.2853,  0.7035,  0.5321,  0.0775, -0.1770,  0.4480, -0.3498,
           0.3491],
         [ 0.3119,  0.3091,  0.8754,  0.9511,  0.6395,  0.3721, -0.3617,
          -0.1339],
         [    nan,     nan,     nan,     nan,     nan,     nan,     nan,
              nan],
         [    nan,     nan,     nan,     nan,     nan,     nan,     nan,
              nan],
         [    nan,     nan,     nan,     nan,     nan,     nan,     nan,
              nan],
         [    nan,     nan,     nan,     nan,     nan,     nan,     nan,
              nan],
         [    nan,     nan,     nan,     nan,     nan,     nan,     nan,
              nan],
         [    nan,     nan,     nan,     nan,     nan,     nan,     nan,
              nan],
         [    nan,     nan,     nan,     nan,     nan,     nan,     nan,
              nan]]], grad_fn=<TransposeBackward1>)


RNN 셀은 두 개의 입력을 리턴하는데, 첫번째 리턴값은 모든 시점(timesteps)의 은닉 상태들이며, 두번째 리턴값은 마지막 시점(timestep)의 은닉 상태입니다. 우선 첫번째 리턴값에 대해서 크기를 확인해봅시다.

In [8]:
print(_status.shape) # 최종 time-step의 hidden_state
print(_status)

torch.Size([2, 1, 8])
tensor([[[nan, nan, nan, nan, nan, nan, nan, nan]],

        [[nan, nan, nan, nan, nan, nan, nan, nan]]], grad_fn=<StackBackward>)


# 5. 양방향 순환 신경망(Bidirectional Recurrent Neural Network)

즉, RNN이 과거 시점(time step)의 데이터들을 참고해서, 찾고자하는 정답을 예측하지만 실제 문제에서는 과거 시점의 데이터만 고려하는 것이 아니라 향후 시점의 데이터에 힌트가 있는 경우도 많습니다. 그래서 이전 시점의 데이터뿐만 아니라, 이후 시점의 데이터도 힌트로 활용하기 위해서 고안된 것이 양방향 RNN입니다.


양방향 RNN은 하나의 출력값을 예측하기 위해 기본적으로 두 개의 메모리 셀을 사용합니다. 첫번째 메모리 셀은 앞에서 배운 것처럼 앞 시점의 은닉 상태(Forward States)를 전달받아 현재의 은닉 상태를 계산합니다. 위의 그림에서는 주황색 메모리 셀에 해당됩니다. 두번째 메모리 셀은 앞에서 배운 것과는 다릅니다. 앞 시점의 은닉 상태가 아니라 뒤 시점의 은닉 상태(Backward States)를 전달 받아 현재의 은닉 상태를 계산합니다. 위의 그림에서는 초록색 메모리 셀에 해당됩니다. 그리고 이 두 개의 값 모두가 출력층에서 출력값을 예측하기 위해 사용됩니다.

물론, 양방향 RNN도 다수의 은닉층을 가질 수 있습니다. 아래의 그림은 양방향 순환 신경망에서 은닉층이 1개 더 추가되어 은닉층이 2개인 깊은(deep) 양방향 순환 신경망의 모습을 보여줍니다.


다른 인공 신경망 모델들도 마찬가지이지만, 은닉층을 무조건 추가한다고 해서 모델의 성능이 좋아지는 것은 아닙니다. 은닉층을 추가하면, 학습할 수 있는 양이 많아지지만 또한 반대로 훈련 데이터 또한 그만큼 많이 필요합니다.

양방향 순환 신경망을 파이토치로 구현할 때는 nn.RNN()의 인자인 bidirectional에 값을 True로 전달하면 됩니다. 이번에는 층이 2개인 깊은 순환 신경망이면서 양방향인 경우, 앞서 실습했던 임의의 입력에 대해서 출력이 어떻게 달라지는지 확인해봅시다.

In [9]:
# (batch_size, time_steps, input_size)
inputs = torch.Tensor(1, 10, 5)
cell = nn.RNN(input_size = 5, hidden_size = 8, num_layers = 2, batch_first=True, bidirectional = True)
outputs, _status = cell(inputs)
print(outputs.shape) # (배치 크기, 시퀀스 길이, 은닉 상태의 크기 x 2)

torch.Size([1, 10, 16])


In [12]:
print(outputs)
print(inputs)

tensor([[[ 0.4547, -0.7498,  0.0523, -0.2831, -0.6725,  0.8431, -0.5190,
          -0.3853, -0.0682,  0.4796, -0.1314,  0.9342,  0.7007, -0.5508,
           0.4057, -0.7191],
         [ 0.3338, -0.6785,  0.1591,  0.7569,  0.2374, -0.1649,  0.4449,
           0.2644,  0.1707,  0.7575,  0.3074, -0.9180, -0.1367,  0.2137,
          -0.3925,  0.9701],
         [ 0.5800, -0.7683,  0.5543,  0.0507, -0.1640,  0.4725, -0.3506,
          -0.1252, -0.4136,  0.1818, -0.1966,  0.8233,  0.8917, -0.3287,
           0.7454, -0.3849],
         [ 0.4622, -0.7433,  0.3676,  0.7567,  0.2650, -0.2363,  0.5954,
           0.2526,  0.3062,  0.7207,  0.3140, -0.9290,  0.0895,  0.2181,
          -0.3389,  0.9546],
         [ 0.6717, -0.5481,  0.3836, -0.4985, -0.6615,  0.7911, -0.0148,
          -0.4593, -0.0508,  0.5283, -0.1351,  0.9361,  0.6996, -0.5913,
           0.4180, -0.6946],
         [ 0.3521, -0.7748,  0.2511,  0.8712,  0.2058, -0.2657,  0.5508,
           0.2609,  0.2856,  0.7349,  0.2987, -0.928

첫번째 리턴값의 크기는 단뱡 RNN 셀 때보다 은닉 상태의 크기의 값이 두 배가 되었습니다. 여기서는 (배치 크기, 시퀀스 길이, 은닉 상태의 크기 x 2)의 크기를 가집니다. 이는 양방향의 은닉 상태 값들이 연결(concatenate)되었기 때문입니다.

In [13]:
print(_status.shape) # (층의 개수 x 2, 배치 크기, 은닉 상태의 크기)

torch.Size([4, 1, 8])


두번째 리턴값의 크기는 (층의 개수 x 2, 배치 크기, 은닉 상태의 크기)를 가집니다. 이는 정방향 기준으로는 마지막 시점에 해당되면서, 역방향 기준에서는 첫번째 시점에 해당되는 시점의 출력값을 층의 개수만큼 쌓아 올린 결과값입니다.

